In [1]:
import rdkit

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [2]:
df = pd.read_csv('acetylcholinesterase_03_bioactivity_data_curated.csv')
df

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,7300.0,intermediate
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,100000.0,inactive
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,100000.0,inactive
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,9500.0,intermediate
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,100000.0,inactive
...,...,...,...,...
658,CHEMBL4453659,CCCc1n[nH]c(C(N)=O)c1NC(=O)/C=C/c1ccco1,1020.0,intermediate
659,CHEMBL4638841,Cc1cn([C@H]2C[C@H](n3cc(COc4ccc(S(N)(=O)=O)cc4...,5200.0,intermediate
660,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,6000.0,intermediate
661,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,9100.0,intermediate


In [3]:
df_no_smiles = df.drop(columns='canonical_smiles')


In [4]:
smiles = []

for i in df.canonical_smiles.tolist():
    cpd = str(i).split('.')
    cpd_longest = max(cpd, key = len)
    smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')


In [5]:
df_clean_smiles = pd.concat([df_no_smiles,smiles], axis=1)
df_clean_smiles

,molecule_chembl_id,standard_value,class,canonical_smiles
0,CHEMBL314057,7300.0,intermediate,O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NCCCCCCCCCCC...
1,CHEMBL266842,100000.0,inactive,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...
2,CHEMBL314847,100000.0,inactive,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...
3,CHEMBL86984,9500.0,intermediate,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...
4,CHEMBL87554,100000.0,inactive,N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H](Cc1c[nH]c...
...,...,...,...,...
658,CHEMBL4453659,1020.0,intermediate,CCCc1n[nH]c(C(N)=O)c1NC(=O)/C=C/c1ccco1
659,CHEMBL4638841,5200.0,intermediate,Cc1cn([C@H]2C[C@H](n3cc(COc4ccc(S(N)(=O)=O)cc4...
660,CHEMBL4632724,6000.0,intermediate,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...
661,CHEMBL4637598,9100.0,intermediate,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...


# Calculate Lipinski descriptors
Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

The Lipinski's Rule stated the following:

- Molecular weight < 500 Dalton
-Octanol-water partition coefficient (LogP) < 5
-Hydrogen bond donors < 5
-Hydrogen bond acceptors < 10

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [6]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [7]:
df_lipinski = lipinski(df_clean_smiles.canonical_smiles)
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,682.198,8.37560,4.0,5.0
1,922.417,6.35110,8.0,9.0
2,594.045,6.12970,4.0,6.0
3,775.324,8.46130,4.0,7.0
4,822.300,4.78510,8.0,8.0
...,...,...,...,...
658,288.307,1.70600,3.0,4.0
659,478.487,-0.81598,3.0,11.0
660,506.497,-1.03798,3.0,12.0
661,505.513,-1.46498,4.0,11.0


In [8]:
df_combined = pd.concat([df,df_lipinski], axis=1)


In [9]:
df_combined

,molecule_chembl_id,canonical_smiles,standard_value,class,MW,LogP,NumHDonors,NumHAcceptors
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,7300.0,intermediate,682.198,8.37560,4.0,5.0
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,100000.0,inactive,922.417,6.35110,8.0,9.0
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,100000.0,inactive,594.045,6.12970,4.0,6.0
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,9500.0,intermediate,775.324,8.46130,4.0,7.0
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,100000.0,inactive,822.300,4.78510,8.0,8.0
...,...,...,...,...,...,...,...,...
658,CHEMBL4453659,CCCc1n[nH]c(C(N)=O)c1NC(=O)/C=C/c1ccco1,1020.0,intermediate,288.307,1.70600,3.0,4.0
659,CHEMBL4638841,Cc1cn([C@H]2C[C@H](n3cc(COc4ccc(S(N)(=O)=O)cc4...,5200.0,intermediate,478.487,-0.81598,3.0,11.0
660,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,6000.0,intermediate,506.497,-1.03798,3.0,12.0
661,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,9100.0,intermediate,505.513,-1.46498,4.0,11.0


# Convert IC50 to pIC50
To allow IC50 data to be more uniformly distributed, we will convert IC50 to the negative logarithmic scale which is essentially -log10(IC50).

This custom function pIC50() will accept a DataFrame as input and will:

Take the IC50 values from the standard_value column and converts it from nM to M by multiplying the value by 10$^{-9}$
Take the molar value and apply -log10
Delete the standard_value column and create a new pIC50 column

In [10]:
import numpy as np

def pIC50(input):
    pIC50 = []

    for i in input['standard_value_norm']:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input.drop('standard_value_norm', 1)
        
    return x

Point to note: Values greater than 100,000,000 will be fixed at 100,000,000 otherwise the negative logarithmic value will become negative.


df_combined.standard_value.describe()


In [11]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop('standard_value', 1)
        
    return x

In [12]:
df_norm = norm_value(df_combined)
df_norm

,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,standard_value_norm
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,intermediate,682.198,8.37560,4.0,5.0,7300.0
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,inactive,922.417,6.35110,8.0,9.0,100000.0
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,inactive,594.045,6.12970,4.0,6.0,100000.0
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,intermediate,775.324,8.46130,4.0,7.0,9500.0
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,inactive,822.300,4.78510,8.0,8.0,100000.0
...,...,...,...,...,...,...,...,...
658,CHEMBL4453659,CCCc1n[nH]c(C(N)=O)c1NC(=O)/C=C/c1ccco1,intermediate,288.307,1.70600,3.0,4.0,1020.0
659,CHEMBL4638841,Cc1cn([C@H]2C[C@H](n3cc(COc4ccc(S(N)(=O)=O)cc4...,intermediate,478.487,-0.81598,3.0,11.0,5200.0
660,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,intermediate,506.497,-1.03798,3.0,12.0,6000.0
661,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,intermediate,505.513,-1.46498,4.0,11.0,9100.0


In [13]:
df_final = pIC50(df_norm)
df_final


,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
0,CHEMBL314057,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,intermediate,682.198,8.37560,4.0,5.0,5.136677
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,inactive,922.417,6.35110,8.0,9.0,4.000000
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,inactive,594.045,6.12970,4.0,6.0,4.000000
3,CHEMBL86984,CC(C)(C)OC(=O)N1CCC[C@@H]1C(=O)N[C@H](Cc1c[nH]...,intermediate,775.324,8.46130,4.0,7.0,5.022276
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,inactive,822.300,4.78510,8.0,8.0,4.000000
...,...,...,...,...,...,...,...,...
658,CHEMBL4453659,CCCc1n[nH]c(C(N)=O)c1NC(=O)/C=C/c1ccco1,intermediate,288.307,1.70600,3.0,4.0,5.991400
659,CHEMBL4638841,Cc1cn([C@H]2C[C@H](n3cc(COc4ccc(S(N)(=O)=O)cc4...,intermediate,478.487,-0.81598,3.0,11.0,5.283997
660,CHEMBL4632724,Cc1cn([C@H]2C[C@H](n3cc(COC(=O)c4ccc(S(N)(=O)=...,intermediate,506.497,-1.03798,3.0,12.0,5.221849
661,CHEMBL4637598,Cc1cn([C@H]2C[C@H](n3cc(CNC(=O)c4ccc(S(N)(=O)=...,intermediate,505.513,-1.46498,4.0,11.0,5.040959


In [14]:
df_final.to_csv('acetylcholinesterase_04_bioactivity_data_3class_pIC50.csv')


In [16]:
df_2class = df_final[df_final['class'] != 'intermediate']
df_2class

,molecule_chembl_id,canonical_smiles,class,MW,LogP,NumHDonors,NumHAcceptors,pIC50
1,CHEMBL266842,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,inactive,922.417000,6.3511,8.0,9.0,4.000000
2,CHEMBL314847,CC(C)(C)OC(=O)N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,inactive,594.045000,6.1297,4.0,6.0,4.000000
4,CHEMBL87554,CCN(CC)CC.N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[C@H]...,inactive,822.300000,4.7851,8.0,8.0,4.000000
5,CHEMBL421541,CC(C)(C)OC(=O)N[C@H](Cc1cc2ccccc2[nH]1)C(=O)N[...,inactive,864.421000,9.6791,6.0,7.0,4.000000
6,CHEMBL313345,CCN(CC)CC.O=C(N[C@H](Cc1c[nH]c2ccccc12)C(=O)NC...,inactive,675.207000,6.8138,5.0,6.0,4.000000
...,...,...,...,...,...,...,...,...
650,CHEMBL3948172,CC1CCCC2(OC(O)C=CC2=O)O1,active,198.218000,0.7456,1.0,4.0,7.899629
652,CHEMBL4076246,C/C(=C\C(=O)Nc1cc([123I])ccc1C(=O)O)c1ccc2cccc...,inactive,453.268589,5.1846,2.0,2.0,4.551603
653,CHEMBL4081286,O=c1cc(-c2cc(O)c(O)cc2[123I])oc2c(O)c(O)ccc12,active,408.136589,2.8870,4.0,6.0,6.013721
654,CHEMBL4062047,C/C(=C\C(=O)Nc1ccc([123I])cc1C(=O)O)c1ccc2cccc...,inactive,453.268589,5.1846,2.0,2.0,4.521578


In [19]:
df_2class.to_csv('acetylcholinesterase_05_bioactivity_data_2class_pIC50.csv')


Exploratory Data Analysis (Chemical Space Analysis) via Lipinski descriptors


In [18]:
!pip install seaborn

  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)


In [17]:
import seaborn as sns
sns.set(style='ticks')
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [40]:
str1.swapcase()

'IQRA ayESHA'

In [13]:
str1.startswith('qra',1,3)

False